# Huggingface SageMaker-SDK - T5 Language Modeling example

## Introduction

このnotebookはSageMakerのHuggingFaceコンテナを使用して、言語モデル（ここではT5を想定）の学習を行います。   

以下を参考にしています
- https://github.com/huggingface/transformers/tree/master/examples/flax/language-modeling

- https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part7.html

- https://github.com/megagonlabs/t5-japanese

- https://arxiv.org/abs/1910.10683

- https://github.com/google-research/text-to-text-transfer-transformer#gpu-usage

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install tensorflow-datasets==4.4.0

In [ ]:
!pip install transformers tokenizers datasets

In [ ]:
!pip install jax>=0.2.8
!pip install jaxlib>=0.1.59
!pip install flax>=0.3.5
!pip install optax>=0.0.9
!pip install torch==1.10.0+cpu torchvision==0.11.1+cpu torchaudio==0.10.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip install sentencepiece==0.1.96

configファイルを言語モデル用のディレクトリに保存する

In [ ]:
from transformers import T5Config

config = T5Config.from_pretrained("google/t5-v1_1-base", vocab_size=32000)
config.save_pretrained("./src/japanese-t5-base")

前工程で作成したTokenizerが動作するか確認する

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("./src/japanese-t5-base")
tokenizer.tokenize("私は元気です。あなたは元気ですか？")

このサンプルノートブックではwiki40bを使用します。

In [ ]:
# https://note.com/npaka/n/n0a2d0a4b806e
import os
import tensorflow_datasets as tfds

ds = tfds.load('wiki40b/ja', split='train', try_gcs=True)

# データセットをテキスト形式で出力する関数
def create_txt(file_name, tf_data):
    start_paragraph = False

    # ファイルの書き込み
    with open(file_name, 'w') as f:
         for wiki in tf_data.as_numpy_iterator():
                for text in wiki['text'].decode().split('\n'):
                    if start_paragraph:
                        text = text.replace('_NEWLINE_', '') # _NEWLINE_は削除
                        f.write(text + '\n')
                        start_paragraph = False
                    if text == '_START_PARAGRAPH_': # _START_PARAGRAPH_のみ取得
                        start_paragraph = True

# データセットをテキスト形式で出力
create_txt('wiki_40b.txt', ds)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('text', data_files='wiki_40b.txt', cache_dir="./")

In [ ]:
dataset

In [ ]:
dataset['train'][0]

データをS3へアップロードします。

In [ ]:
#import botocore
from datasets.filesystems import S3FileSystem
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
dataset['train'].to_json('train.json', force_ascii=False)

s3_prefix = 'samples/datasets/wiki40b-jp'

input_train = sess.upload_data(
    path='train.json', 
    key_prefix=f'{s3_prefix}/train'
)
print(input_train)

## Starting Sagemaker Training Job

`HuggingFace`のトレーニングジョブを作成するためにはHuggingFace Estimatorが必要になります。
Estimatorは、エンドツーエンドのAmazonSageMakerトレーニングおよびデプロイタスクを処理します。 Estimatorで、どのスクリプトをentry_pointとして使用するか、どのinstance_typeを使用するか、どのhyperparametersを渡すかなどを定義します。

```python
huggingface_estimator = HuggingFace(
    entry_point='run_t5_mlm_flax.py',
    source_dir='./src',
    instance_type='ml.p4d.24xlarge',
    instance_count=1,
    transformers_version='4.11',
    #pytorch_version='1.9',
    tensorflow_version='2.5',
    py_version='py37',
    role=role,
    hyperparameters=hyperparameters,
)
```

SageMakerトレーニングジョブを作成すると、SageMakerはhuggingfaceコンテナを実行するために必要なec2インスタンスの起動と管理を行います。
`run_t5_mlm_flax.py`をアップロードし、sagemaker_session_bucketからコンテナ内の/opt/ml/input/dataにデータをダウンロードして、トレーニングジョブを実行します。

HuggingFace estimatorで定義したhyperparametersは、名前付き引数として渡されます。

またSagemakerは、次のようなさまざまな環境変数を通じて、トレーニング環境に関する有用なプロパティを提供しています。

- `SM_MODEL_DIR`：トレーニングジョブがモデルアーティファクトを書き込むパスを表す文字列。トレーニング後、このディレクトリのアーティファクトはモデルホスティングのためにS3にアップロードされます。
- `SM_NUM_GPUS`：ホストで使用可能なGPUの数を表す整数。
- `SM_CHANNEL_XXXX`：指定されたチャネルの入力データを含むディレクトリへのパスを表す文字列。たとえば、HuggingFace estimatorのfit呼び出しでtrainとtestという名前の2つの入力チャネルを指定すると、環境変数SM_CHANNEL_TRAINとSM_CHANNEL_TESTが設定されます。

このトレーニングジョブをローカル環境で実行するには、`instance_type='local'`、GPUの場合は`instance_type='local_gpu'`で定義できます（GPUの場合は追加で設定が必要になります[SageMakerのドキュメント](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)を参照してください）。

**Note：これはSageMaker Studio内では機能しません**

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'model_type':'t5',
    'train_file': '/opt/ml/input/data/train/train.json',
    #'validation_file': '/opt/ml/input/data/validation/dev.csv',
    #'test_file': '/opt/ml/input/data/test/test.csv',
    'config_name':'./japanese-t5-base',
    'tokenizer_name': './japanese-t5-base',
    'max_seq_length': 512,
    'per_device_train_batch_size': 32,
    'per_device_eval_batch_size': 32,
    'adafactor': 'True',
    'learning_rate': 0.001,
    'weight_decay': 0.001,
    'warmup_steps': 100,
    'overwrite_output_dir': 'True',
    'preprocessing_num_workers': 96,
    'num_train_epochs': 1, 
    #'logging_strategy':  'epoch',
    #'save_strategy': 'epoch',
    #'evaluation_strategy': 'epoch',
    'logging_steps': 200,
    'save_steps': 500,
    'eval_steps': 500,
    'output_dir':'/opt/ml/model',
    'push_to_hub':'False'
}

ここではサンプルのため、学習は1 epochのみとします。ml.p4d.24xlargeで30min程度で完了します。

In [ ]:
#distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [ ]:
# estimator
huggingface_estimator = HuggingFace(
    role=role,
    entry_point='run_t5_mlm_flax.py',
    source_dir='./src',
    instance_type='ml.p4d.24xlarge',
    instance_count=1,
    max_run=60*60*24*5,
    volume_size=500,
    transformers_version='4.11',
    #pytorch_version='1.9',
    tensorflow_version='2.5',
    py_version='py37',
    hyperparameters=hyperparameters,
    distribution=None
)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': input_train})

## Download-model-from-s3

In [ ]:
import os

OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
from sagemaker.s3 import S3Downloader

# 学習したモデルのダウンロード
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path='.', # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

In [ ]:
# OUTPUT_DIRに解凍します

!tar -zxvf model.tar.gz -C output

## (Optional) Convert from Flax Model to PyTorch Model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, FlaxT5ForConditionalGeneration

mdl_path = "./output"

pt_model = T5ForConditionalGeneration.from_pretrained(mdl_path, from_flax=True)
pt_model.save_pretrained(mdl_path)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(mdl_path)